In [ ]:
# default_exp model

# Model

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
# export
import pytorch_lightning as pl

from transformers import EncoderDecoderModel, AutoTokenizer

In [ ]:
# export
class RetroFitModel(pl.LightningModule):
    def __init__(
        self,
        encoder_name,
        decoder_name,
        column,
        weight_decay=0.1
        lr=5e-4,
        freeze_decoder=True
    ):
        self.encoder_tokenizer = AutoTokenizer.from_pretrained(encoder_name)
        self.decoder_tokenizer = AutoTokenizer.from_pretrained(decoder_name)
        self.column = column
        self.weight_decay = weight_decay
        self.lr = lr
        self.lr_scheduler_type = lr_scheduler_type
        self.freeze_decoder = freeze_decoder
        self.model = EncoderDecoderModel.from_encoder_decoder_pretrained(encoder_name, decoder_name)

    def training_step(self, batch, batch_idx):
        retrieved = batch["retrieved_examples"]
        retrieved = [self.encoder_tokenizer.eos_token.join(r) for r in retrieved]
        input_ids = self.encoder_tokenizer(retrieved, return_tensors="pt").input_ids
        labels = self.decoder_tokenizer(batch[self.column], return_tensors="pt").input_ids
        loss = self.model(input_ids=input_ids, labels=input_ids).loss

        self.log("trn_loss", loss, on_step=True, on_epoch=True, logger=True)
        return loss
    
    def get_grouped_params(self, model, no_decay=["bias", "LayerNorm.weight"]):
        params_with_wd, params_without_wd = [], []
        for n, p in model.named_parameters():
            if any(nd in n for nd in no_decay):
                params_without_wd.append(p)
            else:
                params_with_wd.append(p)
        return [
            {"params": params_with_wd, "weight_decay": self.weight_decay},
            {"params": params_without_wd, "weight_decay": 0.0},
        ]

    def configure_optimizers(self, skip_special_tokens=True):
        # Prepare the optimizer and learning rate scheduler
        param_model = self.model if not freeze_decoder else self.model.encoder
        optimizer = AdamW(self.get_grouped_params(param_model), lr=self.lr)
        return optimizer

    def generate(self, retrieved_examples):
        retrieved = [self.encoder_tokenizer.eos_token.join(r) for r in retrieved_examples]
        input_ids = self.encoder_tokenizer(retrieved, return_tensors="pt").input_ids
        generated = self.model.generate(input_ids, decoder_start_token_id=self.model.config.decoder.bos_token_id)

        return self.decoder_tokenizer.decode(generated, skip_special_tokens=skip_special_tokens)

In [ ]:
# hide
from nbdev.export import notebook2script

notebook2script()